In [3]:
#Task 1: Retrieving and Preparing the Data (5%)
#Being a careful data scientist, you know that it is vital to set the goal of the project, then
#thoroughly pre-process any available data (each attribute) before starting to analyse and model
#it. In your report in Task 4, You need to clearly state the goal of your project, and the design/steps of
#pre-processing your data. Please ensure you understand the data you selected.

In [4]:
#import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import datetime as dt
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import seaborn as sns
from sklearn.cluster import Birch
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, validation_curve
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score,r2_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import DBSCAN

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
#import retail file
data = pd.read_excel('online_retail_II.xlsx', sheet_name="Year 2009-2010")

In [ ]:
#display data
data

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
data = data.dropna()
data

In [ ]:
data.info()

In [ ]:
#check format data types
data.dtypes

We can see that the involve date column represents valid date-time type format

In [ ]:
data['Country'].value_counts().sort_index()

In [ ]:
#we can see the min value is 0.0 and max value is 38970.0
data['Price'].value_counts().sort_index()

In [ ]:
pd.set_option("display.max_rows", None)
data['Description'].value_counts().sort_index()

In [ ]:
data['Description'] = data['Description'].str.replace(' ', '')
data['Description'] = data['Description'].str.replace("'", '')
data['Description'] = data['Description'].str.replace(",", '')
data['Description'] = data['Description'].str.replace("/", '')
data['Description'].value_counts().sort_index()

In [ ]:
data["TotalPrice"] = data["Price"] * data["Quantity"]
pd.set_option("display.max_rows", 200)

In [ ]:
data

In [ ]:
data['Customer ID'].value_counts().sort_index()

Since there are some customer ID with weird value(s), we drop them.

In [ ]:
#drop nan values
data['Customer ID'] = data['Customer ID'].dropna()
data

In [ ]:
data.isna().sum()

In [ ]:
#Data visualization
# Count of transactions by country
sns.set_style('darkgrid')
plt.figure(figsize=(15, 8))
sns.countplot(x='Country', data=data)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Calculate order per month
data['month']= data['InvoiceDate'].dt.month
data['year']= data['InvoiceDate'].dt.year
data['month_year'] = pd.to_datetime(data[['year', 'month']].assign(DAY=1))
data['revenue'] = data['Price'] * data['Quantity']
data['month_year'].value_counts().sort_index()

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10,5))
plot = pd.DataFrame(data.groupby(['month_year'])['Invoice'].count()).reset_index()
ax = sns.lineplot(x="month_year", y="Invoice", data = plot)
ax.set_title('Number of orders per month')
plt.show()


In [ ]:
#Calculate revenue per month
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10,5))
plot = pd.DataFrame(data.groupby(['month_year'])['revenue'].sum()).reset_index()
ax = sns.lineplot(x="month_year", y="revenue", data = plot)
ax.set_title('Revenue per month')
plt.show()

We can se that after peaked at 2010, the revenue and also number of order per month decrease gradually

In [ ]:
#check duplication for the Customer ID and stocl code column 
groupdt = data.groupby(['Customer ID', 'StockCode']).size().reset_index(name='counts')
groupdt
#discplay the groupdt data frame 
groupdt[groupdt['counts'] > 1]

In [ ]:
analysisdata = data[['Invoice',"TotalPrice","InvoiceDate"]].groupby([data.InvoiceDate.dt.year,data.InvoiceDate.dt.month,"Invoice"]).sum()
analysisdata

In [ ]:
analysisdata2 = data[['Invoice',"TotalPrice","InvoiceDate"]].groupby([data.InvoiceDate.dt.year,data.InvoiceDate.dt.month]).sum()
analysisdata2

In [ ]:
analysisdata2.plot.bar(y='TotalPrice')

In [ ]:
data.hist(figsize = (15,15))


In [ ]:
plt.figure(figsize= (9,8))
cor = data.corr() 
sns.heatmap(cor, annot=True, cmap=plt.cm.RdYlBu)
plt.show()

In [ ]:
#group customer id and data
CusPri = data.groupby('Customer ID')['TotalPrice'].sum()
CusPri = CusPri.reset_index()
CusPri

In [ ]:
#group by the customer id with the total amount that they spent
#sort the values in descending order
InCus = data.groupby('Customer ID')['Invoice'].count()
InCus = InCus.reset_index()
InCus


In [ ]:
#merge data frames 
customers = pd.merge(CusPri, InCus, on='Customer ID')
customers


Also, we can get more details 
In this task, I observes which customer has and has not visited the store.

In [ ]:
# find the most recent date the store active
last= data['InvoiceDate'].max()
last


In [ ]:
data['Last Active']=last - data['InvoiceDate']
data

In [ ]:
#find the difference between the last date and the date of the invoice
recent = data.groupby('Customer ID')['Last Active'].min()
recent = recent.reset_index()
recent['Last Active']= recent['Last Active'].dt.days
recent

In [ ]:
#group all 
customers = pd.merge(customers, recent, on='Customer ID')
customers

In [ ]:
X = customers['TotalPrice'].quantile(0.05)
Y = customers['TotalPrice'].quantile(0.95)
iqr = Y - X

In [ ]:
#Use the quartile variable to remove outliers
customers = customers[(customers['TotalPrice'] >= X - 1.5*iqr) & (customers['TotalPrice'] < Y + 1.5*iqr)]


In [ ]:
#After removing outliers, we can scale the data 
scc = StandardScaler()
scale= scc.fit_transform(customers[['Invoice','TotalPrice','Last Active']])
scale.shape

In [ ]:
scaleValue = pd.DataFrame(scale, columns=['Invoice','TotalPrice','Last Active'])
scaleValue

KMean Cluster
We need to provide K value and we can find the K value using elbow graph

In [ ]:
target = []
for i in range(2,7):
    kmeans = KMeans(n_clusters=i, max_iter=1000, random_state=42, n_init=50, verbose = True).fit(scaleValue)
    target.append(kmeans.inertia_)  

In [ ]:
print(len(target)) # 5

In [ ]:
plt.plot(range(2,7,1),target)
plt.xticks(ticks = range(2,7))

In [ ]:
# create the final model with 3 clusters 
kmeans = KMeans(n_clusters=3, max_iter=1000, random_state=42, n_init=50, verbose = True).fit(scaleValue)
cluster_labels = kmeans.labels_
customers['Cluster'] = cluster_labels
customers


In [ ]:
sns.boxplot(x='Cluster', y='TotalPrice', data=customers)
plt.show()

In [ ]:
sns.boxplot(x='Cluster', y='Invoice', data=customers)

In [ ]:
sns.boxplot(x='Cluster', y='Last Active', data=customers)

In [ ]:
#calculate the silhouette score
def plot_silhouette_score(data, labels):
    cluster_labels = np.unique(labels)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(data, labels, metric='euclidean')
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    
    for i, c in enumerate(cluster_labels):
        c_silhouette_vals = silhouette_vals[labels == c]
        c_silhouette_vals.sort()
        y_ax_upper += len(c_silhouette_vals)
        color = cm.jet(i / n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, edgecolor='none', color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2)
        y_ax_lower += len(c_silhouette_vals)
    
    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color="red", linestyle="--")
    plt.yticks(yticks, cluster_labels + 1)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.show()

    return silhouette_avg
    
plot_silhouette_score(scaleValue, cluster_labels)

In [ ]:
silhouette = dict()
for k in range(2, 15):
    kmeans = KMeans(init='k-means++', n_clusters=k)
    clusters = kmeans.fit_predict(scaleValue)
    silhouette[k] = silhouette_score(scaleValue, clusters)

In [ ]:
plt.plot (list(silhouette.keys()), list(silhouette.values()))
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()


In [ ]:
print('The best number of clusters is: ', max(silhouette, key=silhouette.get))

In [ ]:
plot_silhouette_score(scaleValue, customers['Cluster'])

In [ ]:
pd.DataFrame(pd.Series(kmeans.labels_).value_counts(), columns=['Cluster Size'])

In [ ]:
# clustering technique 2: DBSCAN
#generate the model
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
dbscan.fit(scaleValue)

In [ ]:
#find the number of clusters
labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters

In [ ]:
#find the number of noise points
n_noise = list(labels).count(-1)
n_noise

In [ ]:
#calculate the silhouette score
silhouette= silhouette_score(scaleValue, labels)
silhouette

In [ ]:
print("estimate number of clusters: %d" % n_clusters)
print("Silhouette Coefficient: %0.3f" % silhouette_score(scaleValue, labels))

the silhouette score for kmeans clustering 0.515263490702985
the silhouette score for DBSCAN 0.5789194190984394


## Classification


In [ ]:
#Splitting train and test set 
#Support vector classifier
X = customers[['Invoice','TotalPrice','Last Active']]
y = customers['Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def train_and_plot(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # classification report
    print('\n{}'.format(classification_report(y_test, y_pred)))
    print('Accuracy score: ', accuracy_score(y_test, y_pred))
    print('Confusion matrix: \n', confusion_matrix(y_test, y_pred))
    print('Classification report: \n', classification_report(y_test, y_pred))
    plot_confusion_matrix(model, X_test, y_test, cmap='Blues')
    plt.show()

In [ ]:
# Naive Bayes
# from sklearn.naive_bayes import GaussianNB (choose model class)
nb_model = GaussianNB() # instantiate model
nb_model.fit(X_train, y_train) #fit the model to the data

In [ ]:
predictions = nb_model.predict(X_test)
acc = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (acc * 100.0))

In [ ]:
pipeline = Pipeline([('scaler', StandardScaler()), ('estimator', SVC())])
param = {'estimator__C': [0.1, 1, 10, 100, 1000], 'estimator__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'estimator__kernel': ['rbf']}
grid = GridSearchCV(pipeline, param_grid=param, cv=5, verbose=2)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.score(X_test, y_test))


## Linear regression 

In [ ]:
#fold cross validation 
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = grid, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
# create test and train
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_train.shape)

In [ ]:
lrm = linear_model.LinearRegression()
lrm_model = lrm.fit(X_train, y_train)
lrm_prediction = lrm.predict(X_test)

In [ ]:
lrm_prediction[0:10]

In [ ]:
print(lrm_prediction)
type(lrm_prediction)

In [ ]:
df2 = pd.DataFrame(X_test)
df2["revenue actual"] = y_test
df2["revenue pre"] = lrm_prediction
df2.head(10)

In [ ]:
plt.scatter (y_test, lrm_prediction) 
plt.title ("Plot of Predicted vs. Actual Values") 
plt.xlabel ('revenue actual')
plt.ylabel ('revenue pre')
print('Accuracy: ', lrm_model.score(X_test, y_test) * 100)

## Multiple linear regression

In [ ]:
X = data.drop(["revenue", "Description", "InvoiceDate", "Country", "month_year", "StockCode", "Invoice"], axis = 1).values
y = data["revenue"].values

In [ ]:
# split to test and train set
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size = 0.3, random_state = 0)
ml = LinearRegression()
ml.fit(X_train, y_train)

In [ ]:
y_pred = ml.predict(X_test)
plt.figure(figsize = (10,5))
plt.scatter(y_test, y_pred)
plt.xlabel("actual")
plt.ylabel("predict")
print("Accuracy: ", r2_score(y_pred, y_test) * 100)

In [ ]:
pred_y_df = pd.DataFrame({'Actual revenue': y_test, 'Predicted revenue': y_pred, 'Difference': y_test-y_pred})
pred_y_df [0:20]